In [ ]:
# import library
import pandas as pd
import numpy as np 
import matlab 
import matplotlib as plt
import os
import numpy as np


In [10]:
# read the file
# mortality
zipped = True
if zipped:
    mdf_path = r'data/mdf_final.csv'
    mdf = pd.read_csv(mdf_path, low_memory=False)
    mdf = mdf.drop(columns='Unnamed: 0')
    
    pdf_path = r'data\pdf_final.csv'
    pdf = pd.read_csv(pdf_path, low_memory=False)
    pdf = pdf.drop(columns=['Lb', 'Unnamed: 0'])
    
    lnmrdf = pd.read_csv(r'data/lnmr_final.csv', low_memory=False)
    lnmrdf = lnmrdf.drop(columns='Unnamed: 0')
    
    
else:
    mortality_path_7 = r'data/MortIcd7.csv'
    mortality_data_7 = pd.read_csv(mortality_path_7, low_memory=False)

    mortality_path_8 = r'data/MortIcd8.csv'
    mortality_data_8 = pd.read_csv(mortality_path_8, low_memory=False)

    mortality_path_9 = r'data/Morticd9.csv'
    mortality_data_9 = pd.read_csv(mortality_path_9, low_memory=False)

    mortality_path_2002 = r'data/Morticd10_part1.csv'
    mortality_data_2002 = pd.read_csv(mortality_path_2002, low_memory=False)

    mortality_path_2007 = r'data/Morticd10_part2.csv'
    mortality_data_2007 = pd.read_csv(mortality_path_2007, low_memory=False)

    mortality_path_2012 = r'data/Morticd10_part3.csv'
    mortality_data_2012 = pd.read_csv(mortality_path_2012, low_memory=False)

    mortality_path_2016 = r'data/Morticd10_part4.csv'
    mortality_data_2016 = pd.read_csv(mortality_path_2016, low_memory=False)

    mdf = pd.concat([mortality_data_7, mortality_data_8, mortality_data_9, mortality_data_2002, mortality_data_2007, mortality_data_2012, mortality_data_2016], ignore_index=True)
    undesired_mortality_columns = ['Admin1', 'SubDiv', 'Sex', 'List', 'Frmat', 'IM_Frmat', 'Cause', 'IM_Deaths1','IM_Deaths2', 'IM_Deaths3', 'IM_Deaths4'] 
    mdf = mdf.drop(columns = undesired_mortality_columns)

    # population
    pop_path = r'data/pop.csv'
    pdf = pd.read_csv(pop_path, low_memory=False)
    dropped = False
    if not dropped:
        undesired_population_columns = ['Admin1', 'SubDiv', 'Sex', 'Frmat', 'Lb'] 
        pdf = pdf.drop(columns = undesired_population_columns)

# country code
country_code_path = r'data/country_codes.csv'
country_code = pd.read_csv(country_code_path, low_memory=False)
# creating the country code map 
country_code_map = {}
for index, rows in country_code.iterrows():
    country_code_map[rows['country']] = rows['name']

In [11]:
# needs to swap out the nah columns into averages 
done = True
if not done:
    not_used_country_list = []
    country_num_count = 0
    not_useful = False
    for key in country_code_map:
        country_num_count += 1
        display(country_num_count / len(country_code_map))
        country_mdf = mdf[mdf['Country'] == key]
        
        if country_mdf.empty:
            not_used_country_list.append(key)
        
        else:
            min_year = int(country_mdf['Year'].min())
            max_year = int(country_mdf['Year'].max()) 
            for i in range (min_year, max_year + 1):
                
                if not_useful:
                    break
                
                mask = (mdf['Country'] == key) & (mdf['Year'] == i)
                curr_Year = i

                for c in mdf.columns.values:
                    
                    if not_useful:
                        break
                    
                    average = mdf.loc[mask, c].mean()   
                    
                    max_iter = max_year - curr_Year
                    count = 0 
                    while np.isnan(average):
                        curr_Year += 1
                        count += 1
                        mask_alt = (mdf['Country'] == key) & (mdf['Year'] == curr_Year) 
                        average = mdf.loc[mask_alt, c].mean()   
                        
                        if count > max_iter:
                            not_useful = True
                            break 
                        
                        
                    mdf.loc[mask, c] = mdf.loc[mask, c].fillna(average)
                    curr_Year = i 
                    
            if not_useful:
                display(key)
                not_used_country_list.append(key)
                index = mdf[mdf['Country'] == key].index
                mdf = mdf.drop(index)
                not_useful = False
                
    undesired_mortality_columns = ['Admin1', 'SubDiv', 'Sex', 'List', 'Frmat', 'IM_Frmat', 'Cause', 'IM_Deaths1','IM_Deaths2', 'IM_Deaths3', 'IM_Deaths4'] 
    mdf = mdf.drop(columns = undesired_mortality_columns)            
    mdf = mdf.groupby(['Country', 'Year'], as_index=True).agg("sum").reset_index()
    
    # building the potential country list 
    country_list = list(set(mdf['Country']))
    country_list.sort()
    len(country_list) 
        
            

        

In [12]:
# build pdf
done = True
if not done:
    not_used_pop_list = []
    country_num_count = 0
    not_useful = False
    for key in country_list:
        
            country_num_count += 1
            display(country_num_count / len(country_list))
            
            country_pdf = pdf[pdf['Country'] == key]
            
            if country_pdf.empty:
                not_used_pop_list.append(key)
            
            else:
                min_year = int(country_pdf['Year'].min())
                max_year = int(country_pdf['Year'].max()) 
                
                if min_year == max_year:
                    if (country_pdf.isnull().sum().sum() > 0):
                        not_useful = True
                    else:
                        continue
                
                for i in range (min_year, max_year + 1):
                    
                    if not_useful:
                        break
                    
                    mask = (pdf['Country'] == key) & (pdf['Year'] == i)
                    curr_Year = i

                    for c in pdf.columns.values:
                        
                        if not_useful:
                            break
                        
                        average = pdf.loc[mask, c].mean()   
                        
                        max_iter = max_year - curr_Year
                        count = 0 
                        while np.isnan(average):
                            curr_Year += 1
                            count += 1
                            mask_alt = (pdf['Country'] == key) & (pdf['Year'] == curr_Year)
                            average = pdf.loc[mask_alt, c].mean()   
                            
                            if count > max_iter:
                                not_useful = True
                                break 
                            
                            
                        pdf.loc[mask, c] = pdf.loc[mask, c].fillna(average)
                        curr_Year = i 
                        
                if not_useful:
                    display(key)
                    not_used_pop_list.append(key)
                    index = pdf[pdf['Country'] == key].index
                    pdf = pdf.drop(index)
                    not_useful = False
                    
    # delete the unused countries 
    for key in not_used_pop_list:
        country_list.remove(key)
        
    # finalizing pdf 
    pdf = pdf[pdf['Country'].isin(country_list)]
    pdf = pdf.groupby(['Country', 'Year'], as_index=True).agg("sum").reset_index()
    pdf.to_csv(r'data/pdf_final.csv')
    
    # finalizing mdf 
    mdf = mdf[mdf['Country'].isin(country_list)]
    mdf.to_csv(r'data/mdf_final.csv')
                    
                
        
    

In [13]:
# create the log mortality rate graph 
from gc import callbacks

done = True
if not done:
    
    lnmr_dict = {}
    lnmr_dict['Country'] = []
    lnmr_dict['Year'] = []
    for i in range (6, 27):
        lnmr_column = f'Log Mortality{i}'
        lnmr_dict[lnmr_column] = []
    
    lnmrdf = pd.DataFrame(lnmr_dict)
    
    country_list = list(set(mdf['Country']))
    country_list.sort()
    
    country_num_count = 0
    for key in country_list:
        
        country_num_count += 1
        display(country_num_count / len(country_list))
        
        country_mdf = mdf[mdf['Country'] == key]
        min_year = int(country_mdf['Year'].min())
        max_year = int(country_mdf['Year'].max()) 
        
        for i in range (min_year, max_year + 1):
            
            m_mask = (mdf['Country'] == key) & (mdf['Year'] == i)
            p_mask = (pdf['Country'] == key) & (pdf['Year'] == i)
            
            if pdf[p_mask].empty or mdf[m_mask].empty:
                continue
            else:
                lnmrdf.loc[lnmrdf.shape[0]] = ['None'] * lnmrdf.shape[1]
                lnmrdf.loc[lnmrdf.shape[0]-1, ['Country']] = key
                lnmrdf.loc[lnmrdf.shape[0]-1, ['Year']] = i
            
            for c in range(6, 27):
                
                if c == 6:
                    deaths = 0
                    population = 0
                    for j in range (2, 7):
                        m_column = f'Deaths{c}'
                        p_column = f'Pop{c}'
                        lnmr_column = f'Log Mortality{c}'
                    try:
                        deaths += mdf[m_mask][m_column].values.astype('float')[0]
                        population += pdf[p_mask][p_column].values.astype('float')[0]
                    except:
                        display(key)
                        display(i)
                        display(c)
                        callbacks()
                else:
                    m_column = f'Deaths{c}'
                    p_column = f'Pop{c}'
                    lnmr_column = f'Log Mortality{c}'
                    try:
                        deaths = mdf[m_mask][m_column].values.astype('float')[0]
                        population = pdf[p_mask][p_column].values.astype('float')[0]
                    except:
                        display(key)
                        display(i)
                        display(c)
                        callbacks()
                try:
                    ratio = deaths / population
                    lnmrdf.loc[lnmrdf.shape[0]-1, lnmr_column] = np.log(ratio)
                except:
                    display(key)
                    display(i)
                    display(c)
                    callbacks()
    
    lnmrdf.to_csv(r'data/lnmr_final.csv')

In [14]:
display(mdf, pdf, lnmrdf)

,Country,Year,Deaths1,Deaths2,Deaths3,Deaths4,Deaths5,Deaths6,Deaths7,Deaths8,...,Deaths17,Deaths18,Deaths19,Deaths20,Deaths21,Deaths22,Deaths23,Deaths24,Deaths25,Deaths26
0,2140,1956,19216,7706.0,2640.0,42.857143,26.488095,28.869048,534.0,228.0,...,520.0,736.0,816.0,824.0,718.0,600.0,804.0,97.432905,30.455076,238.0
1,2140,1957,20942,8310.0,3096.0,42.857143,26.488095,28.869048,610.0,196.0,...,584.0,738.0,848.0,974.0,738.0,680.0,860.0,97.432905,30.455076,194.0
2,2140,1958,18732,7186.0,2558.0,42.857143,26.488095,28.869048,568.0,268.0,...,548.0,688.0,832.0,890.0,768.0,690.0,730.0,97.432905,30.455076,188.0
3,2140,1959,20352,7808.0,2438.0,42.857143,26.488095,28.869048,526.0,230.0,...,764.0,868.0,912.0,1098.0,946.0,760.0,916.0,97.432905,30.455076,234.0
4,2140,1960,20126,8068.0,2674.0,42.857143,26.488095,28.869048,550.0,294.0,...,622.0,820.0,878.0,918.0,816.0,664.0,804.0,97.432905,30.455076,208.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1738,5020,2012,294196,2062.0,162.0,112.000000,66.000000,66.000000,256.0,282.0,...,10936.0,15156.0,19794.0,24492.0,32880.0,47818.0,55770.0,40118.000000,18220.000000,6.0
1739,5020,2013,295356,2188.0,192.0,118.000000,74.000000,48.000000,272.0,272.0,...,11172.0,15218.0,21148.0,24910.0,32672.0,46428.0,55868.0,40728.000000,17876.000000,36.0
1740,5020,2014,307160,2024.0,134.0,90.000000,66.000000,68.000000,272.0,252.0,...,11662.0,15910.0,21650.0,25982.0,33976.0,46418.0,57258.0,44762.000000,19380.000000,68.0
1741,5020,2015,318104,1982.0,154.0,116.000000,64.000000,86.000000,264.0,294.0,...,11956.0,15846.0,22060.0,26964.0,35228.0,46718.0,59546.0,47626.000000,21220.000000,4.0


,Country,Year,Pop1,Pop2,Pop3,Pop4,Pop5,Pop6,Pop7,Pop8,...,Pop17,Pop18,Pop19,Pop20,Pop21,Pop22,Pop23,Pop24,Pop25,Pop26
0,2140,1955,1030000.0,39000.0,152000.0,57000.0,56000.0,55000.0,168000.0,131000.0,...,20000.0,20000.0,12000.0,9000.0,5900.0,3600.0,2500.0,12101.836284,3742.256872,0.0
1,2140,1956,1069000.0,41000.0,159000.0,57000.0,56000.0,55000.0,174000.0,137000.0,...,22000.0,20000.0,12000.0,10000.0,5900.0,3600.0,2500.0,12101.836284,3742.256872,0.0
2,2140,1957,1111000.0,43000.0,165000.0,57000.0,56000.0,55000.0,181000.0,142000.0,...,22000.0,22000.0,13000.0,10000.0,6400.0,3900.0,2700.0,12101.836284,3742.256872,0.0
3,2140,1958,1153100.0,44300.0,171000.0,57000.0,56000.0,55000.0,187800.0,147300.0,...,23100.0,22000.0,13200.0,10200.0,6500.0,3900.0,2800.0,12101.836284,3742.256872,0.0
4,2140,1959,1200000.0,46100.0,177900.0,57000.0,56000.0,55000.0,195400.0,153300.0,...,24000.0,23000.0,13700.0,10700.0,6700.0,4000.0,2900.0,12101.836284,3742.256872,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,5020,2017,24594202.0,306218.0,319302.0,314197.0,314901.0,319952.0,1588064.0,1473503.0,...,1507880.0,1332357.0,1195119.0,957490.0,675976.0,468494.0,307205.0,143558.000000,39818.000000,0.0
1916,5020,2018,24966643.0,302580.0,307291.0,320155.0,315940.0,317762.0,1604857.0,1517847.0,...,1533335.0,1359441.0,1209828.0,1017163.0,697153.0,484461.0,308873.0,147445.000000,42002.000000,0.0
1917,5020,2019,25340217.0,302441.0,303351.0,309380.0,321322.0,318044.0,1616624.0,1561095.0,...,1553519.0,1393636.0,1230577.0,1060915.0,728940.0,503783.0,311718.0,151207.000000,44227.000000,0.0
1918,5020,2020,25655289.0,296660.0,301934.0,304257.0,310846.0,322116.0,1619875.0,1603137.0,...,1566970.0,1439153.0,1262567.0,1110579.0,767379.0,526142.0,315992.0,156374.000000,46278.000000,0.0


,Country,Year,Log Mortality6,Log Mortality7,Log Mortality8,Log Mortality9,Log Mortality10,Log Mortality11,Log Mortality12,Log Mortality13,...,Log Mortality17,Log Mortality18,Log Mortality19,Log Mortality20,Log Mortality21,Log Mortality22,Log Mortality23,Log Mortality24,Log Mortality25,Log Mortality26
0,2140,1956,-7.552318,-5.786415,-6.398391,-6.205039,-5.539923,-5.450509,-5.266569,-4.999585,...,-3.744969,-3.302257,-2.688248,-2.496170,-2.106238,-1.791759,-1.134447,-4.821948,-4.811191,inf
1,2140,1957,-7.552318,-5.692793,-6.585468,-6.052552,-5.606360,-5.432665,-5.285895,-5.003518,...,-3.628897,-3.394854,-2.729824,-2.328929,-2.160109,-1.746639,-1.144075,-4.821948,-4.811191,inf
2,2140,1958,-7.552318,-5.801011,-6.309240,-5.972046,-5.846797,-5.485288,-5.480339,-5.146126,...,-3.741313,-3.465009,-2.764140,-2.438922,-2.135768,-1.732040,-1.344330,-4.821948,-4.811191,inf
3,2140,1959,-7.552318,-5.917503,-6.502073,-6.155645,-5.879673,-5.558848,-5.384149,-5.049198,...,-3.447241,-3.277058,-2.709511,-2.276753,-1.957620,-1.660731,-1.152450,-4.821948,-4.811191,inf
4,2140,1960,-7.552318,-5.903626,-6.287403,-6.264640,-5.791174,-5.646624,-5.432453,-5.226524,...,-3.781702,-3.252452,-2.870949,-2.427364,-2.134862,-1.844558,-1.316768,-4.821948,-4.811191,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1684,5020,2012,-8.407946,-8.620694,-8.504059,-7.285969,-7.051856,-6.904555,-6.696214,-6.393050,...,-4.827657,-4.391491,-3.945724,-3.428934,-2.857857,-2.232448,-1.612353,-1.022693,-0.466430,inf
1685,5020,2013,-8.741400,-8.584719,-8.545236,-7.367627,-7.099435,-6.981511,-6.732161,-6.410505,...,-4.826360,-4.401208,-3.934265,-3.446580,-2.893935,-2.265345,-1.640264,-1.073316,-0.522129,inf
1686,5020,2014,-8.421919,-8.612926,-8.623826,-7.528668,-7.097038,-6.946706,-6.653617,-6.333623,...,-4.801355,-4.375574,-3.944889,-3.445261,-2.885807,-2.267231,-1.639425,-1.047470,-0.470648,inf
1687,5020,2015,-8.189480,-8.668914,-8.476008,-7.342110,-7.119793,-6.953597,-6.674782,-6.314799,...,-4.795069,-4.393388,-3.959283,-3.451100,-2.884723,-2.260969,-1.619843,-1.038237,-0.459403,inf


In [ ]:
# fit the model 
# construct the matrix M, of log mortalit rate changes, second year to last year
# time span will be 5 years, since the age group is divided in 5 
 

In [ ]:
# looping through the code map to locate each different countries
# condense the mortality and pop rate 
undesired = ['Admin1', 'SubDiv', 'Sex', 'List', 'Frmat', 'IM_Frmat', 'Cause', 'IM_Deaths1','IM_Deaths2', 'IM_Deaths3', 'IM_Deaths4']
modified_dict = {}
for i in mdf.columns.values:
    if i not in undesired:
       modified_dict[i] = []

modified_df = pd.DataFrame(modified_dict)
list_delete_keys = []
consider_population = True

for key in country_code_map: 
    temp_mortality_df = mdf[mdf['Country'] == key] # filter to the country
    temp_pop_data = pop_data[pop_data['Country'] == key]
    if temp_mortality_df.empty:
        list_delete_keys.append(key)
    else:
        for i in range (1980, 2003):
            temp_temp_mortality_df = temp_mortality_df.loc[temp_mortality_df['Year'] == i] # filter to the year 
            temp_temp_pop_data = temp_pop_data.loc[temp_pop_data['Year'] == i] 
            if temp_temp_mortality_df.empty:
                continue
            else:
                modified_df.loc[modified_df.shape[0]] = ['None'] * modified_df.shape[1]
                modified_df.loc[modified_df.shape[0]-1, ['Country']] = key
                modified_df.loc[modified_df.shape[0]-1, ['Year']] = i
                column_name = modified_df.columns.values 
                k = 1
                for c in column_name[2:]:
                    j = f'Pop{k}'
                    population = temp_temp_pop_data[j].sum()
                    if c in temp_temp_mortality_df.columns.values:
                        if consider_population:
                            if population == 0:
                                modified_df.loc[modified_df.shape[0]-1, [c]] = 0
                            else:
                                modified_df.loc[modified_df.shape[0]-1, [c]] = temp_temp_mortality_df[c].sum() / population
                        else:    
                            modified_df.loc[modified_df.shape[0]-1, [c]] = temp_temp_mortality_df[c].sum() 
                    k += 1
            
                # modified_df.index = modified_df.index + 1
                # modified_df = modified_df.sort_index()      

In [ ]:
# building the mortality model 
country_mortality_model = {}
final_beginning_year = 9999
for key in country_code_map:
    # isolate country
    temp_modified_df = modified_df[modified_df['Country'] == key]
    # temp_modified_df.drop(columns=['Country'], inplace = True) 
    
    # finding alpha 
    last_year = temp_modified_df['Year'].max()
    
    if last_year < final_beginning_year:
        final_beginning_year = last_year
    
    first_year = temp_modified_df['Year'].min()
    duration = last_year - first_year + 1
    last_row = temp_modified_df[temp_modified_df['Year'] == last_year].loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
    first_row = temp_modified_df[temp_modified_df['Year'] == first_year].loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
    aphla = (last_row - first_row) / duration

    country_mortality_model[key] = {'a': aphla, 'd': duration}

In [ ]:
final_ending_year = 2050
current_year = 2000

mortality_pred_all = []
while current_year <= final_ending_year:
    
    mortality_pred_year = {}
    
    for key in country_code_map: 
        
        print(key)
        
        # isolate the data to the target data row \n",
        temp_modified_df = modified_df[(modified_df['Country'] == key)]  
        # check if there is real data for this year for this country\n",
        if current_year <= 2002:
            df = temp_modified_df[temp_modified_df['Year'] == current_year + 1]
            if not df.empty:
                df = df.loc[:, ~df.columns.isin(['Country', 'Year'])].values.astype('float')
                mortality_pred_year[key] = df 
                continue
 
        # clean the data for the row
        temp_modified_df = temp_modified_df[temp_modified_df['Year'] == current_year].loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
        # find the aphla, b and k value for the model for future \n",
        aphla = country_mortality_model[key]['a']
        duration = country_mortality_model[key]['d']
        m_hat = (temp_modified_df - aphla)
        s, u, v = np.linalg.svd(m_hat)
        scalar = s[0]
        k = scalar * u[0]
        b = v[0] / scalar 
  
        # compute the next years result",
        change = np.exp(aphla + b * k)
        display(temp_modified_df.shape)
        display(change.shape)
        next_year = temp_mortality_df + change
        mortality_pred_year[key] = next_year
        
        # update parameters
        duration += 1
        aphla = (aphla * duration + change) / duration
        country_mortality_model[key]['a'] = aphla
        country_mortality_model[key]['d'] = duration
    
    current_year += 1
    mortality_pred_all.append(mortality_pred_year)
        
        


In [ ]:
key = 1125
temp_modified_df = modified_df[(modified_df['Country'] == key)]  
# clean the data for the row
temp_modified_df = temp_modified_df[temp_modified_df['Year'] == current_year].loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
# find the aphla, b and k value for the model for future \n",
aphla = country_mortality_model[key]['a']
duration = country_mortality_model[key]['d']
m_hat = (temp_modified_df - aphla)
s, u, v = np.linalg.svd(m_hat)
scalar = s[0]
k = scalar * u[0]
b = v[0] / scalar 

display(aphla)
display(k, b)
change = np.exp(aphla + b * k)
display(temp_modified_df)
display(change)
display(temp_modified_df.shape)
next_year = temp_mortality_df + change
mortality_pred_year[key] = next_year